In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")

from utils.detection_utils import *
from utils.camera_utils import corner_detection

In [2]:
run_name = "RUN4"

#spiegel_calib = "spiegel1.mp4"
#front_calib = "front.mp4"
#stereo_calib = "stereo.mp4"
stereo_calib = "stereo3.mp4"
stereo_calib2 = "stereo4.mp4"
rows = 8
columns = 10
image_scaling = 3
fallback_manual = True

In [ ]:
rows_inner = rows - 1
columns_inner = columns - 1

In [ ]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(name=run_name, camera_size={0: (480, 240), 1: (480, 240)})
vL = videoLoader()

In [ ]:
sC = stereoCamera()
sC.load_from_yaml(f"{run_name}.yaml")

In [ ]:
def triangulate(SC, point1, point2):
    RT1 = np.concatenate([np.eye(3), [[0], [0], [0]]], axis=-1)
    RT2 = np.concatenate([SC.conf["rotation_matrix"][0], SC.conf["translation_matrix"][0]], axis=-1)

    cm0 = np.array(SC.conf["optimized_camera_matrix"][0])
    cm1 = np.array(SC.conf["optimized_camera_matrix"][1])
    dist0 = np.array(SC.conf["distortion"][0])
    dist1 = np.array(SC.conf["distortion"][0])
    
    P1 = SC.conf["camera_matrix"][0] @ RT1
    P2 = SC.conf["camera_matrix"][1] @ RT2

    point1 = cv2.undistortPoints(point1, cm0, dist0 , None, cm0)[0][0]
    point2 = cv2.undistortPoints(point2, cm1, dist1, None, cm0)[0][0]

    coordinate = cv2.triangulatePoints(P1, P2, point1, point2)
    coordinate = (coordinate[:3, :] / coordinate[3, :]).T[0]

    return coordinate

In [ ]:
file = os.path.join(calibration_dir, stereo_calib)
vL.load_video(file)
frame = vL[170]
frame1, frame2 = sC(frame)
frames1, frames2 = [frame1], [frame2]

In [ ]:
corners1 = corner_detection(frames1, cam=0, image_scaling=image_scaling, rows_inner=rows_inner, columns_inner=columns_inner, fallback_manual=fallback_manual)
corners1 = corner_detection(frames2, cam=1, image_scaling=image_scaling, rows_inner=rows_inner, columns_inner=columns_inner, fallback_manual=fallback_manual)


In [ ]:
corners1, corners2 = show_and_switch(frame1, frame2, corners1, corners2, 8, 10, image_scaling=2)

In [ ]:
coordinates = []

for [point1], [point2] in zip(corners1, corners2):
    coordinates.append(triangulate(sC, point1, point2))

In [ ]:
coordinates

# Redefine the points with the provided smaller subset
points_smaller_subset = coordinates

# Calculate distances between consecutive 3D points for the smaller subset
distances_smaller_subset = [np.linalg.norm(points_smaller_subset[i+1] - points_smaller_subset[i]) for i in range(len(points_smaller_subset) - 1)]
np.array([x for x in distances_smaller_subset if x < 0.01]).mean(), "Distance start - end (diagonaly) =", np.linalg.norm(coordinates[5]- coordinates[-6]), coordinates[0], coordinates[-1]

In [ ]:
np.mean([np.linalg.norm(points_smaller_subset[(7*j)+i]- points_smaller_subset[7+(7*j)+i]) for i in range(7) for j in range(8)])

In [ ]:
# Assuming 'coords' is your numpy array of 3D coordinates
coords = np.array(coordinates)  # Update this line based on your actual data

# Create line connections between sequential points
connections = [[a, b] for a, b in zip(range(len(coords)-1), range(1, len(coords)))]

# Calculate max range to set equal axes
x_range = [coords[:, 0].min(), coords[:, 0].max()]
y_range = [coords[:, 1].min(), coords[:, 1].max()]
z_range = [coords[:, 2].min(), coords[:, 2].max()]

# Find the maximum range
max_range = max(x_range[1] - x_range[0], y_range[1] - y_range[0], z_range[1] - z_range[0])

# Calculate the mid points for each axis
x_mid = sum(x_range) / 2
y_mid = sum(y_range) / 2
z_mid = sum(z_range) / 2

# Set the range for each axis to be the max range centered around the mid point
x_range = [x_mid - max_range / 2, x_mid + max_range / 2]
y_range = [y_mid - max_range / 2, y_mid + max_range / 2]
z_range = [z_mid - max_range / 2, z_mid + max_range / 2]

# Create the figure object
fig = go.Figure()

# Add lines between connected points
for conn in connections:
    fig.add_trace(go.Scatter3d(
        x=[coords[conn[0], 0], coords[conn[1], 0]],
        y=[coords[conn[0], 1], coords[conn[1], 1]],
        z=[coords[conn[0], 2], coords[conn[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name='Line'
    ))

# Add markers for each point
fig.add_trace(go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1],
    z=coords[:, 2],
    mode='markers',
    marker=dict(size=1, color='red'),
    name='Points'
))

# Update plot appearance
fig.update_layout(
    title="3D Line Plot",
    scene=dict(
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        zaxis_title='Z Coordinate',
        xaxis=dict(range=x_range, autorange=False),
        yaxis=dict(range=y_range, autorange=False),
        zaxis=dict(range=z_range, autorange=False),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show the plot
fig.show()

In [ ]:
file = os.path.join(calibration_dir, stereo_calib2)
vL.load_video(file)
frame = vL[150]
frame1, frame2 = sC(frame)

In [ ]:
corners1 = label_corners(np.ascontiguousarray(frame1), optimize=False)
corners2 = label_corners(np.ascontiguousarray(frame2), optimize=False)

In [ ]:


corners1, corners2 = show_and_switch(frame1, frame2, corners1, corners2, 8, 10)

In [ ]:
for [point1], [point2] in zip(corners1, corners2):
    coordinates.append(triangulate(sC, point1, point2))

In [ ]:
# Assuming 'coords' is your numpy array of 3D coordinates
coords = np.array(coordinates)  # Update this line based on your actual data

# Create line connections between sequential points
connections = [[a, b] for a, b in zip(range(len(coords)-1), range(1, len(coords)))]

# Calculate max range to set equal axes
x_range = [coords[:, 0].min(), coords[:, 0].max()]
y_range = [coords[:, 1].min(), coords[:, 1].max()]
z_range = [coords[:, 2].min(), coords[:, 2].max()]

# Find the maximum range
max_range = max(x_range[1] - x_range[0], y_range[1] - y_range[0], z_range[1] - z_range[0])

# Calculate the mid points for each axis
x_mid = sum(x_range) / 2
y_mid = sum(y_range) / 2
z_mid = sum(z_range) / 2

# Set the range for each axis to be the max range centered around the mid point
x_range = [x_mid - max_range / 2, x_mid + max_range / 2]
y_range = [y_mid - max_range / 2, y_mid + max_range / 2]
z_range = [z_mid - max_range / 2, z_mid + max_range / 2]

# Create the figure object
fig = go.Figure()

# Add lines between connected points
for conn in connections:
    fig.add_trace(go.Scatter3d(
        x=[coords[conn[0], 0], coords[conn[1], 0]],
        y=[coords[conn[0], 1], coords[conn[1], 1]],
        z=[coords[conn[0], 2], coords[conn[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2),
        name='Line'
    ))

# Add markers for each point
fig.add_trace(go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1],
    z=coords[:, 2],
    mode='markers',
    marker=dict(size=1, color='red'),
    name='Points'
))

# Update plot appearance
fig.update_layout(
    title="3D Line Plot",
    scene=dict(
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        zaxis_title='Z Coordinate',
        xaxis=dict(range=x_range, autorange=False),
        yaxis=dict(range=y_range, autorange=False),
        zaxis=dict(range=z_range, autorange=False),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show the plot
fig.show()